In [1]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.devices import global_phase
from trajectree.quant_info.noise_models import amplitude_damping
from trajectree.quant_info.circuit import Circuit
from trajectree.trajectory import quantum_channel, trajectory_evaluator
import numpy as np
from scipy import sparse as sp
from matplotlib import pyplot as plt
import cirq
import qsimcirq
import time

from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit import transpile, QuantumCircuit
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.primitives import SamplerResult
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    amplitude_damping_error,
    thermal_relaxation_error,
)

from mqt.bench import BenchmarkLevel, get_benchmark

In [2]:
num_qubits = 1
num_trajectories = 2
noise_probability = np.sqrt(0.5)

qc = QuantumCircuit(num_qubits)
qc.h(0)
# qc.cx(0,1)
# qc.x(0)

In [3]:
# print(dumps(qc))
target_basis = ['h', 's', 'x', 't', 'ry', 'rz', 'cx', 'cz']
transpiled_qc_custom = transpile(qc, basis_gates=target_basis, optimization_level=1)

noise_channel = amplitude_damping_error(noise_probability)

two_qubit_noise_channel = noise_channel.tensor(noise_channel)

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise_channel, target_basis[:-2])
noise_model.add_all_qubit_quantum_error(two_qubit_noise_channel, ["cx", 'cz'])

print(noise_model)
transpiled_qc_custom.draw(output="text")

NoiseModel:
  Basis gates: ['cx', 'cz', 'h', 'id', 'ry', 'rz', 's', 'sx', 't', 'x']
  Instructions with noise: ['s', 'x', 'h', 't', 'rz', 'cz', 'ry', 'cx']
  All-qubits errors: ['h', 's', 'x', 't', 'ry', 'rz', 'cx', 'cz']


┌───┐
q: ┤ H ├
   └───┘

In [4]:

observable = SparsePauliOp("Z" * (num_qubits))

# Use the Sampler primitive for latest API
backend = AerSampler()

noisy_estimator = Estimator(options=dict(backend_options=dict(noise_model=noise_model), run_options=dict(shots=num_trajectories)))

pub = (transpiled_qc_custom, observable)

job = noisy_estimator.run([pub])
result = job.result()
pub_result = result[0]
# print((len(result)))
print(float(pub_result.data.evs))
print(float(pub_result.data.stds))

0.5469181606780275
0.0


In [5]:
qc = Circuit(num_qubits, backend = 'tensor')
qc.qiskit_to_trajectree(transpiled_qc_custom, noise_parameter=noise_probability)
qc.expectation('z'*(num_qubits))
evs, times = qc.perform_trajectree_simulation(num_trajectories)
print("outside of simulation")
evs
# np.mean(evs)
# evs

time taken: 0.016738176345825195
[1.0]
time taken: 0.0018281936645507812
[1.0, 0.5469181606780272]
done with simulations
outside of simulation


[1.0, 0.5469181606780272]